In [1]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

2023-08-22 09:24:01.407556: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-22 09:24:01.434872: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-22 09:24:01.435528: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-22 09:24:01.901688: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# # !pip install matplotlib
# !pip install pydot
from tensorflow.keras.utils import plot_model

In [3]:
import string
import re
from string import digits
import numpy as np
from sklearn.model_selection import train_test_split
# import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.utils import to_categorical, plot_model
from keras.layers import Input, Embedding, LSTM, Dense, dot, concatenate, Activation
import tensorflow as tf
from tensorflow.keras.models import Model

In [4]:
from tensorflow.keras.layers import (Dense,Flatten,SimpleRNN,InputLayer,Conv1D,Bidirectional,GRU,LSTM,BatchNormalization,Dropout,Input, Embedding,TextVectorization)


In [ ]:
# # from google.colab import drive
# # drive.mount('/content/drive')
# !unzip /content/drive/MyDrive/hindi_english_parallel.csv.zip -d "/content/language/"

Archive:  /content/drive/MyDrive/hindi_english_parallel.csv.zip
  inflating: /content/language/hindi_english_parallel.csv  


## DATA PRE_PROCESSING

In [5]:
VOCAB_SIZE=20000
ENGLISH_SEQUENCE_LENGTH=32
HINDI_SEQUENCE_LENGTH=32
EMBEDDING_DIM=300
BATCH_SIZE=64

In [6]:
import pandas as pd
lines=pd.read_csv("hindi_english_parallel.csv")
lines.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [7]:
pd.isnull(lines).sum()
lines.dropna(subset=['hindi', 'english'], inplace=True)
pd.isnull(lines).sum()
# len(lines)
lines=lines.sample(n=20000,random_state=42)
print(lines.shape)


(20000, 2)


In [8]:
# Lowercase all characters
lines.english=lines.english.map(lambda x : x.lower())
lines.hindi=lines.hindi.apply(lambda x: x.lower())

# Remove quotes
lines.english=lines.english.apply(lambda x: re.sub("'", '', x))
lines.hindi=lines.hindi.apply(lambda x: re.sub("'", '', x))
exclude = set(string.punctuation) # Set of all special characters

# Remove all the special characters
lines.english=lines.english.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.hindi=lines.hindi.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove extra spaces
lines.english=lines.english.apply(lambda x: x.strip())
lines.hindi=lines.hindi.apply(lambda x: x.strip())
lines.english=lines.english.apply(lambda x: re.sub(" +", " ", x))
lines.hindi=lines.hindi.apply(lambda x: re.sub(" +", " ", x))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.english=lines.english.apply(lambda x: x.translate(remove_digits))
lines.hindi = lines.hindi.apply(lambda x: re.sub("[२३०८१५७९४६0-9]", "", x))



In [9]:
def retain_hindi_alphabet(text):
    return re.sub(r'[^\u0900-\u097F\s]', '', text)
def remove_words_with_numbers(text):
    return re.sub(r'\b[^a-zA-Z\s]+\b', '', text)
def remove_specific_chars(text):
    return re.sub(r'[ं]', '', text)

# Apply the function to the 'hindi' column
lines.hindi = lines.hindi.apply(retain_hindi_alphabet)
lines.english = lines.english.apply(remove_words_with_numbers)
#Apply the function to the 'hindi' column
lines['hindi'] = lines['hindi'].apply(remove_specific_chars)

In [10]:
lines['length_eng_sentence']=lines['english'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi'].apply(lambda x:len(x.split(" ")))
lines=lines[lines['length_eng_sentence']<=ENGLISH_SEQUENCE_LENGTH]
lines=lines[lines['length_hin_sentence']<=HINDI_SEQUENCE_LENGTH-2]
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])
lines.shape

maximum length of Hindi Sentence  30
maximum length of English Sentence  32


(17589, 4)

In [11]:
# Add start and end tokens to target sequences
lines.hindi = lines.hindi.apply(lambda x : 'starttoken '+ x + ' endtoken')
# lines.english = lines.english.apply(lambda x : 'starttoken '+ x + ' endtoken')
lines.head()

,hindi,english,length_eng_sentence,length_hin_sentence
136823,starttoken कृपया एक भिन्न नाम उपयोग करे endtoken,please use a different name,5,6
783298,starttoken तेज endtoken,agile,1,1
1216571,starttoken एक अन्य परस्पर जुड़ा हुआ पहलू नवान्...,another inter linked aspect is that of innovat...,10,10
166305,starttoken endtoken,attach the information to bug,6,1
1166752,starttoken कुछ राज्यो मे एक ऊपरी सदन है जिसे र...,certain states have an upper house also called...,11,14


In [12]:
output=lines.hindi.copy()
output=output.apply(lambda x: re.sub("starttoken", "", x))
output.head()

136823                 कृपया एक भिन्न नाम उपयोग करे endtoken
783298                                          तेज endtoken
1216571     एक अन्य परस्पर जुड़ा हुआ पहलू नवान्वेषण और उद...
166305                                              endtoken
1166752     कुछ राज्यो मे एक ऊपरी सदन है जिसे राज्य विधान...
Name: hindi, dtype: object

In [13]:
english_vectorize_layer=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=ENGLISH_SEQUENCE_LENGTH
)
hindi_vectorize_layer=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=HINDI_SEQUENCE_LENGTH
)

2023-08-22 09:24:51.096600: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-22 09:24:51.097068: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [14]:
english_vectorize_layer.adapt(lines.english)

In [15]:
hindi_vectorize_layer.adapt(lines.hindi)

In [16]:
lines.hindi=lines.hindi.apply(lambda x: re.sub("endtoken", "", x))
lines.head()

,hindi,english,length_eng_sentence,length_hin_sentence
136823,starttoken कृपया एक भिन्न नाम उपयोग करे,please use a different name,5,6
783298,starttoken तेज,agile,1,1
1216571,starttoken एक अन्य परस्पर जुड़ा हुआ पहलू नवान्...,another inter linked aspect is that of innovat...,10,10
166305,starttoken,attach the information to bug,6,1
1166752,starttoken कुछ राज्यो मे एक ऊपरी सदन है जिसे र...,certain states have an upper house also called...,11,14


In [17]:
def vectorizer(inputs,out):
  return {'input_1':english_vectorize_layer(inputs['english']),
          'input_2':hindi_vectorize_layer(inputs['hindi'])},hindi_vectorize_layer(out)

In [18]:
dataset,output_sol=vectorizer(lines,output)

In [19]:
# output_sol=output_sol.tolist()
print(len(dataset['input_1']), len(output_sol),type(output_sol))

17589 17589 <class 'tensorflow.python.framework.ops.EagerTensor'>


In [20]:
type(output_sol[0])

tensorflow.python.framework.ops.EagerTensor

In [21]:
def data_generator(train, output, batch_size=128):
    while True:
        num_samples = len(output)
        for start_idx in range(0, num_samples, batch_size):
            end_idx = start_idx + batch_size
            batch_input_1 = train['input_1'][start_idx:end_idx]
            batch_input_2 = train['input_2'][start_idx:end_idx]
            batch_output = output[start_idx:end_idx]
            yield (batch_input_1, batch_input_2), batch_output

In [22]:
def valid_generator(train, output):
  batch_input_1 = train['input_1'][0:]
  batch_input_2 = train['input_2'][0:]
  batch_output = output[0:]
  return (batch_input_1, batch_input_2), batch_output

In [23]:
train_size = int(0.30 * len(dataset['input_1']))
train_input_dict = {
    'input_1': dataset['input_1'][:train_size],
    'input_2': dataset['input_2'][:train_size]
}
train_output_list = output_sol[:train_size]

validation_input_dict = {
    'input_1': dataset['input_1'][train_size:],
    'input_2': dataset['input_2'][train_size:]
}
validation_output_list = output_sol[train_size:]

In [24]:
print(len(train_input_dict['input_1']), len(train_output_list),len(validation_output_list))
# NUM_BATCHES=int(lines.shape[0]/BATCH_SIZE)

5276 5276 12313


In [25]:
len_validation=len(validation_output_list)

In [26]:
len_train=len(train_input_dict['input_1'])

In [35]:
### ENCODER
class Encoder(tf.keras.Model):
  def __init__(self,vocab_size,embedding_dim,units):
    super(Encoder,self).__init__()
    self.embedding_dim=embedding_dim
    self.vocab_size=vocab_size
    self.units=units

  def build(self,input_shape):
    self.embedding=Embedding(self.vocab_size,self.embedding_dim)
    self.lstm=LSTM(self.units,return_sequences=True)

  def call(self,x):
    x=self.embedding(x)
    output=self.lstm(x)
    return output

In [36]:
HIDDEN_UNITS=256
EMBEDDING_DIM=256
encoder=Encoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS)
encoder_output=encoder(tf.zeros([128,ENGLISH_SEQUENCE_LENGTH]))
print(encoder_output.shape)

(128, 32, 256)


In [37]:
### ATTENTION
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self,units):
    super(BahdanauAttention,self).__init__()
    self.units=units

  def build(self,input_shape):
    self.w_1=tf.keras.layers.Dense(self.units)
    self.w_2=tf.keras.layers.Dense(self.units)
    self.w=tf.keras.layers.Dense(1)

  def call(self,prev_dec_state,enc_states):
    scores=self.w(
        tf.nn.tanh(
        self.w_1(tf.expand_dims(prev_dec_state,-2)) +
        self.w_2(enc_states)))
    
    attention_weights=tf.nn.softmax(scores,axis=1)
    context_vector=attention_weights*enc_states
    context_vector=tf.reduce_sum(context_vector,axis=1)
    return context_vector,attention_weights

In [38]:
bahdanau_attention=BahdanauAttention(256)
context_vector,attention_weights=bahdanau_attention(tf.zeros([128,32]),tf.zeros([128,ENGLISH_SEQUENCE_LENGTH,32]))
print(context_vector.shape)
print(attention_weights.shape)

(128, 32)
(128, 32, 1)


In [39]:
##DECORDER
class Decoder(tf.keras.Model):
  def __init__(self,vocab_size,embedding_dim,dec_units,sequence_length):
    super(Decoder,self).__init__()
    self.embedding_dim=embedding_dim
    self.vocab_size=vocab_size
    self.dec_units=dec_units
    self.sequence_length=sequence_length
   
  def build(self,input_shape):
    self.dense=Dense(self.vocab_size,activation="softmax")
    self.gru=GRU(
        self.dec_units,return_sequences=True,return_state=True)
    self.attention=BahdanauAttention(self.dec_units)
    self.embedding=Embedding(self.vocab_size,self.embedding_dim)

  def call(self,x,hidden,shifted_target):
    outputs=[]
    context_vectors=[]
    attention_weightss=[]
    shifted_target=self.embedding(shifted_target)
    
    for t in range(self.sequence_length):
      context_vector,attention_weights=self.attention(hidden,x)
      dec_input=context_vector+shifted_target[:,t]
      output,hidden=self.gru(tf.expand_dims(dec_input,1))
      outputs.append(output[:,0])

    outputs=tf.convert_to_tensor(outputs)
    outputs=tf.transpose(outputs, perm=[1,0,2])

    outputs=self.dense(outputs)
    return outputs,attention_weights

In [40]:
decoder=Decoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS,HINDI_SEQUENCE_LENGTH)
outputs,attention_weights=decoder(encoder_output,tf.zeros([128,HIDDEN_UNITS]),tf.zeros([128,HINDI_SEQUENCE_LENGTH]))
print(outputs.shape)
print(attention_weights.shape)

(128, 32, 20000)
(128, 32, 1)


In [41]:
### ENCODER
input = Input(shape=(ENGLISH_SEQUENCE_LENGTH,), dtype="int64", name="input_1")
encoder=Encoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS)
encoder_output=encoder(input)

### DECODER
shifted_target=Input(shape=(HINDI_SEQUENCE_LENGTH,), dtype="int64", name="input_2")
decoder=Decoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS,HINDI_SEQUENCE_LENGTH)
decoder_output,attention_weightss=decoder(encoder_output,tf.zeros([1,HIDDEN_UNITS]),shifted_target)

### OUTPUT
bahdanau=Model([input,shifted_target],decoder_output)
bahdanau.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32)]                 0         []                            
                                                                                                  
 encoder_1 (Encoder)         (None, 32, 256)              5645312   ['input_1[0][0]']             
                                                                                                  
 input_2 (InputLayer)        [(None, 32)]                 0         []                            
                                                                                                  
 decoder_1 (Decoder)         ((None, 32, 20000),          1078659   ['encoder_1[0][0]',           
                              (None, 32, 1))              3          'input_2[0][0]']         

In [73]:
plot_model(bahdanau,show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [42]:
class BLEU(tf.keras.metrics.Metric):
    def __init__(self,name='bleu_score'):
        super(BLEU,self).__init__()
        self.bleu_score=0

    def update_state(self,y_true,y_pred,sample_weight=None):
      y_pred=tf.argmax(y_pred,-1)
      self.bleu_score=0
      for i,j in zip(y_pred,y_true):
        tf.autograph.experimental.set_loop_options()

        total_words=tf.math.count_nonzero(i)
        total_matches=0
        for word in i:
          if word==0:
            break
          for q in range(len(j)):
            if j[q]==0:
              break
            if word==j[q]:
              total_matches+=1
              j=tf.boolean_mask(j,[False if y==q else True for y in range(len(j))])
              break

        self.bleu_score+=total_matches/total_words

    def result(self):
        return self.bleu_score/BATCH_SIZE

In [43]:
bahdanau.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(5e-4),
    metrics=[BLEU()],
    run_eagerly=True)

In [44]:
checkpoint_filepath = 'nmt_lstm_attention.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
    save_best_only=True,)

In [ ]:
train_generator = data_generator(train_input_dict, train_output_list, BATCH_SIZE)
validation_generator = data_generator(validation_input_dict, validation_output_list,BATCH_SIZE)

validation_data=validation_generator,
history=bahdanau.fit(
    train_generator,
    epochs=20,
    callbacks=[model_checkpoint_callback],verbose=1)


Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: not enough values to unpack (expected 2, got 0)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: not enough values to unpack (expected 2, got 0)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
   1325/Unknown - 5768s 4s/step - loss: 2.2772 - bleu: 0.3094

In [ ]:
index_to_word={x:y for x, y in zip(range(len(hindi_vectorize_layer.get_vocabulary())),
                                   hindi_vectorize_layer.get_vocabulary())}

In [1]:
def translator(english_sentence):
  tokenized_english_sentence=english_vectorize_layer([english_sentence])
  shifted_target='starttoken'

  for i in range(FRENCH_SEQUENCE_LENGTH):
    tokenized_shifted_target=french_vectorize_layer([shifted_target])
    output=bahdanau.predict([tokenized_english_sentence,tokenized_shifted_target])
    french_word_index=tf.argmax(output,axis=-1)[0][i].numpy()
    current_word=index_to_word[french_word_index]
    if current_word=='endtoken':
      break
    shifted_target+=' '+current_word
  return shifted_target[11:]

In [ ]:
translator('the novel was published in book form in')

1/1 [==============================] - 0s 94ms/step


'[UNK] के लिए [UNK] के लिए [UNK]'

In [ ]:
word_to_index={y:x for x, y in zip(range(len(hindi_vectorize_layer.get_vocabulary())),
                                   hindi_vectorize_layer.get_vocabulary())}

In [ ]:
seq2seq_gru.evaluate(validation_generator)

11/11 [==============================] - 17s 1s/step - loss: 2.1099


2.109869956970215